# Примеры

## Пример 1, обучение RandomForest выявлению сепсиса

### Устройство MIMIC
В примерах разберем, как с помощью модуля sickness-screening обучить модель предсказывать сепсис на наборе данных MIMIC.
В MIMIC есть множество таблиц, но для примера нам потребуются следующие таблицы:
1. **chartevents.csv** —— содержит данные мониторинга пациентов, например: температура тела, артериальное давление.
2. **labevents.csv** —— содержит данные различных анализов пациентов, например различные характеристики анализов крови для пациентов.
3. **diagnoses.csv** —— содержит информацию о диагнозах, которые получил пациент.
4. **d_icd_diagnoses** —— расшифрофки кодов диагнозов для каждого диагноза.
5. **d_labitems.csv** —— расшифрофки кодов анализов для каждого пациента. 

### Установим библиотеку

In [3]:
!pip3 install sickness-screening

  Preparing metadata (setup.py) ... done
  Created wheel for sickness-screening: filename=sickness_screening-1.0.3-py3-none-any.whl size=21919 sha256=7d7c8c3bfe185b7afd77eeb2c658bb4d216fc07cd6ac31e4afc4df266fb7735b
  Stored in directory: /Users/artemij/Library/Caches/pip/wheels/17/da/cf/354fd644d3b12984a940a63a0d11d97238e310c61521a358d3
Successfully built sickness-screening


#### Аггрегирование данных о диагнозах пациентов:

##### Импортирование библиотеки

In [4]:
import sickness_screening as ss

#### Аггрегирование данных о диагнозах пациентов:

In [6]:
df_gotten_diagnoses = ss.get_diagnoses_data(patient_diagnoses_csv='diagnoses.csv', 
                 all_diagnoses_csv='d_icd_diagnoses.csv',
                 output_file_csv='gottenDiagnoses.csv')
print(df_gotten_diagnoses.head(30))

    subject_id                                         long_title
0     10000032                                Portal hypertension
1     10000032                                      Other ascites
2     10000032      Cirrhosis of liver without mention of alcohol
3     10000032  Unspecified viral hepatitis C without hepatic ...
4     10000032  Chronic airway obstruction, not elsewhere clas...
5     10000032                      Bipolar disorder, unspecified
6     10000032                      Posttraumatic stress disorder
7     10000032                    Personal history of tobacco use
8     10000032    Unspecified viral hepatitis C with hepatic coma
9     10000032                                      Other ascites
10    10000032                      Thrombocytopenia, unspecified
11    10000032                  Hyposmolality and/or hyponatremia
12    10000032  Chronic airway obstruction, not elsewhere clas...
13    10000032      Cirrhosis of liver without mention of alcohol
14    1000

Здесь мы для каждого пациента из **patient_diagnoses_csv** получаем коды диагнозов, а далее, используя **all_diagnoses_csv** 
мы уже получаем файл **output_file_csv** в котором для каждого пациента уже хранится расшифровка его диагноза. 

#### Получение данных о том, есть ли конкретный диагноз у пациента

In [7]:
df_sepsis_info = ss.get_diseas_info(diagnoses_csv='gottenDiagnoses.csv', title_column='long_title', diseas_str='sepsis',
                    diseas_column='has_sepsis', subject_id_column='subject_id', log_stats=True,
                    output_csv='sepsis_info.csv')
print(df_sepsis_info.head(30))

Всего пациентов: 193256
Всего пациентов с сепсисом: 12616
     subject_id  has_sepsis
0      10000032       False
41     10000068       False
42     10000084       False
54     10000108       False
56     10000117       False
78     10000248       False
84     10000280       False
85     10000560       False
88     10000635       False
91     10000719       False
97     10000764       False
116    10000826       False
131    10000826        True
147    10000883       False
152    10000886       False
153    10000904       False
154    10000935       False
236    10000980       False
376    10001176       False
393    10001186       False
416    10001217       False
434    10001319       False
443    10001338       False
465    10001401       False
511    10001401        True
569    10001472       False
581    10001492       False
585    10001663       False
587    10001667       False
600    10001725       False


Здесь используем таблицу, которую мы получили из предыдущего примера, чтобы на выходе получить таблицу, в которой содержатся данные о том,
был ли у этого человека в диагнозе продстрока sepsis, или нет. 

#### Аггрегирование данных, необходимых для нахождения ССВР (синдром системной воспалительной рекции)

Теперь соберем некоторые данные, необходимые для определения ССВР:

In [12]:
df_ssir = ss.get_analyzes_data(analyzes_csv='chartevents.csv', subject_id_col='subject_id', itemid_col='itemid',
                      charttime_col='charttime', value_col='value', valuenum_col='valuenum',
                      itemids=[220045, 220210, 223762, 223761, 225651], rest_columns=['Heart rate', 'Respiratory rate', 'Temperature Fahrenheit', 'Temperature Celsius',
                        'Direct Bilirubin', 'Heart rate_valueuom', 'Respiratory rate_valueuom',
                        'Temperature Fahrenheit_valueuom', 'Temperature Celsius_valueuom', 'Direct Bilirubin_valueuom'], output_csv='ssir.csv')
print(df_ssir.head(30))

    subject_id            charttime Heart rate Respiratory rate  \
0     10000032  2180-07-23 14:00:00        NaN              NaN   
1     10000032  2180-07-23 14:12:00         91               24   
2     10000032  2180-07-23 14:30:00         93               21   
3     10000032  2180-07-23 15:00:00         94               23   
4     10000032  2180-07-23 16:00:00        105               21   
5     10000032  2180-07-23 17:00:00         97               20   
6     10000032  2180-07-23 18:00:00        100               21   
7     10000032  2180-07-23 19:00:00         97               16   
8     10000032  2180-07-23 20:00:00        100               19   
9     10000032  2180-07-23 21:00:00         94               22   
10    10000032  2180-07-23 22:00:00         94               20   
11    10000980  2189-06-27 08:54:00        NaN               23   
12    10000980  2189-06-27 08:56:00         77              NaN   
13    10000980  2189-06-27 09:00:00        NaN               2

Здесь мы используя таблицу **analyzes_csv**, **itemids**(коды анализов, которые мы хотим собрать), **rest_columns**(колонки, которые мы хотим оставить в выходной таблице),
Фунукция собирает из analyzes_csv замеры для пациентов с кодами **itemids** и записать их в **output_csv**, оставив только колонки, которые есть в **rest_columns**
В данной функции **subject_id_col** и **itemid_col** отвечают за колонки, отведенные под коды пациентов и анализов соответсвенно.
**charttime_col** отвечает за время. **valuenum_col** отвечает за колонку с единицами измерения анализов. 

#### Комбинирование данных о диагнозах и ССВР

In [13]:
df_combined = ss.combine_data(first_data_csv='gottenDiagnoses.csv', 
                              second_data_csv='ssir.csv',
                              output_file='diagnoses_and_ssir.csv')
print(df_combined.head(30))

Correct number of patients with sepsis: 12616
Unique patients with sepsis predictions: 9751
Unique patients without sepsis predictions: 41163
Patients with both sepsis and no sepsis records: 6
Total unique patients: 50920
    subject_id            charttime  Heart rate  Respiratory rate  \
0     10000032  2180-07-23 14:00:00         NaN               NaN   
1     10000032  2180-07-23 14:12:00        91.0              24.0   
2     10000032  2180-07-23 14:30:00        93.0              21.0   
3     10000032  2180-07-23 15:00:00        94.0              23.0   
4     10000032  2180-07-23 16:00:00       105.0              21.0   
5     10000032  2180-07-23 17:00:00        97.0              20.0   
6     10000032  2180-07-23 18:00:00       100.0              21.0   
7     10000032  2180-07-23 19:00:00        97.0              16.0   
8     10000032  2180-07-23 20:00:00       100.0              19.0   
9     10000032  2180-07-23 21:00:00        94.0              22.0   
10    10000032  218

#### Сбор и комбинирование данных об анализах крови, с данными об диагнозах и ССВР
Соберем данные об анализах крови пациентов и скомбинируем их в одну таблицу:

In [ ]:
df_merged = ss.merge_and_get_data(merge_with='diagnoses_and_ssir.csv', 
                      blood_csv='labevents.csv',
                      get_data_from='chartevents.csv',
                      output_csv='merged_data.csv',
                      analyzes_names = {
                        51222: "Hemoglobin",
                        51279: "Red Blood Cell",
                        51240: "Large Platelets",
                        50861: "Alanine Aminotransferase (ALT)",
                        50878: "Asparate Aminotransferase (AST)",
                        225651: "Direct Bilirubin",
                        50867: "Amylase",
                        51301: "White Blood Cells"})
print(df_merged.head(30))

/Users/artemij/miniconda3/lib/python3.12/site-packages/sickness_screening/merge_and_get_data.py:65: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  diagnoses_and_ssir = pd.read_csv(merge_with)


Данная функция ищет данные об **analyzes_names** пациентов из таблиц **blood_csv.csv** и **get_data_from**, 
комбинирует их вместе с **merge_with**. Стоит отметить, что эта функция также комбинирует данные о болезни каждого пациента. 

#### Балансировка данных внутри каждого пациента:

In [ ]:
df_balanced = ss.balance_on_patients(balancing_csv='merged_data.csv', disease_col='has_sepsis', subject_id_col='subject_id',
                        output_csv='balance.csv',
                        output_filtered_csv='balance_filtered.csv',
                        filtering_on=200,
                        number_of_patient_selected=50000,
                        log_stats=True
                        )
print(df_balanced.head(30))

#### Компрессия данных о каждом пациенте (если в наборе данных пропуски, то внутри каждого пациента пропуски заполнятся значением из этого пациента)
Теперь заполним пропуски имеющимися данными для каждого пациента, не заполняя статистическими значениями или константами:

In [ ]:
df_compressed = ss.compress(df_to_compress='balanced_data.csv', 
            subject_id_col='subject_id',
            output_csv='compressed_data.csv')

#### Выбрать лучших пациентов с данными для балансировки

In [ ]:
df_choose = ss.choose(compressed_df_csv='compressed_data.csv', 
          output_file='final_balanced_data.csv')

#### Заполнение пропущенных значений модой

In [ ]:
ss.fill_values(balanced_csv='final_balanced_data.csv', 
               strategy='most_frequent', 
               output_csv='filled_data.csv')

#### Тренировка модели на наборе данных

In [ ]:
import sickness_screening as ss
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
model = ss.train_model(df_to_train_csv='filled_data.csv', 
                       categorical_col=['Large Platelets'], 
                       columns_to_train_on=['Amylase'], 
                       model=RandomForestClassifier(),
                       single_cat_column='White Blood Cells', 
                       has_disease_col='has_sepsis', 
                       subject_id_col='subject_id', 
                       valueuom_col='valueuom', 
                       scaler=MinMaxScaler(), 
                       random_state=42, 
                       test_size=0.2)

В этой функции мы обучаем **RandomForestClassifier** из scikit-learn на наборе данных с одной категориальной колонкой, с одной числовой колонкой
и с одной категориальной колонкой, которую можно преобразовать в числовую. В качестве метода нормализации используется **MinMaxScaler** из scikit-learn.

#### Например, можно вставить такие модели, как CatBoostClassifier или SVC с разными ядрами
CatBoostClassifier:

In [ ]:
class_weights = {0: 1, 1: 15}
clf = CatBoostClassifier(loss_function='MultiClassOneVsAll', class_weights=class_weights, iterations=50, learning_rate=0.1, depth=5)
clf.fit(X_train, y_train)

SVC с использованием гауссова ядра с радиальной базовой функцией (RBF):

In [ ]:
class_weights = {0: 1, 1: 13}
param_dist = {
    'C': reciprocal(0.1, 100),
    'gamma': reciprocal(0.01, 10),
    'kernel': ['rbf']
}

svm_model = SVC(class_weight=class_weights, random_state=42)
random_search = RandomizedSearchCV(
    svm_model,
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring=make_scorer(recall_score, pos_label=1),
    n_jobs=-1
)